In [ ]:
# =========================
# BLOCK 0- COLAB SETUP (Install)
# =========================
!pip -q install pymupdf openai tqdm pandas requests

import os, re, json, glob, time
from typing import Dict, Any, Optional, Tuple, List

import fitz  # PyMuPDF
import pandas as pd
import requests
from tqdm import tqdm
from openai import OpenAI


In [ ]:
# =========================
# BLOCK (1) GATHER & EXTRACT PAPERS
# =========================

# --- In Colab, put PDFs here ---
PDF_DIR = "/content/pdfs"  # create this folder and upload PDFs

MAX_PAGES_TO_READ = 10
MAX_CHARS_PER_PAPER = 14000

# Create folder if missing
os.makedirs(PDF_DIR, exist_ok=True)

# =========================
# PDF extraction helpers
# =========================
SECTION_HEADERS = ["abstract","introduction","methods","materials and methods","participants","results","discussion","conclusion","references"]

def _normalize(s: str) -> str:
    s = s.replace("\u00ad", "")
    s = re.sub(r"[ \t]+", " ", s)
    s = re.sub(r"\n{3,}", "\n\n", s)
    return s.strip()

def extract_text_from_pdf(pdf_path: str, max_pages: int) -> str:
    doc = fitz.open(pdf_path)
    pages = []
    for i in range(min(max_pages, doc.page_count)):
        page = doc.load_page(i)

        t = page.get_text("text").strip()
        if t:
            pages.append(t)
            continue

        blocks = page.get_text("blocks")
        block_text = "\n".join(
            b[4] for b in blocks
            if len(b) > 4 and isinstance(b[4], str) and b[4].strip()
        ).strip()
        if block_text:
            pages.append(block_text)

    doc.close()
    return _normalize("\n".join(pages))

def extract_title_from_pdf(pdf_path: str) -> str:
    """
    Heuristic title extraction:
    - Uses the first page
    - Takes the first non-empty line with sufficient length
    - Avoids author lists / affiliations when possible
    """
    try:
        doc = fitz.open(pdf_path)
        page = doc.load_page(0)

        text = page.get_text("text")
        doc.close()

        if not text:
            return "UNKNOWN_TITLE"

        lines = [l.strip() for l in text.splitlines() if l.strip()]

        for line in lines[:10]:
            if len(line) > 20 and not re.search(r"(university|department|email|@)", line, re.I):
                return line

        return lines[0] if lines else "UNKNOWN_TITLE"

    except Exception:
        return "UNKNOWN_TITLE"

def find_section_span(text: str, start_kw: str) -> Optional[Tuple[int, int]]:
    start_pat = re.compile(rf"(?im)^\s*{re.escape(start_kw)}\s*[:\n]", re.MULTILINE)
    m = start_pat.search(text)
    if not m:
        return None
    start = m.end()
    other = [h for h in SECTION_HEADERS if h != start_kw.lower()]
    end_pat = re.compile(r"(?im)^\s*(" + "|".join(map(re.escape, other)) + r")\s*[:\n]", re.MULTILINE)
    m2 = end_pat.search(text[start:])
    end = start + m2.start() if m2 else len(text)
    return start, end

def grab(text: str, header: str) -> str:
    span = find_section_span(text, header)
    if not span:
        return ""
    s, e = span
    return _normalize(text[s:e])

def extract_AMR(text: str) -> Dict[str, str]:
    abstract = grab(text, "Abstract")
    methods = grab(text, "Methods") or grab(text, "Materials and Methods") or grab(text, "Participants")
    results = grab(text, "Results")
    return {"abstract": abstract, "methods": methods, "results": results}

def build_model_input(sections: Dict[str, str], raw_text: str) -> Tuple[str, str]:
    methods = sections.get("methods", "").strip()
    results = sections.get("results", "").strip()
    abstract = sections.get("abstract", "").strip()

    if methods or results:
        parts = []
        if methods:
            parts.append("Methods:\n" + methods)
        if results:
            parts.append("Results:\n" + results)
        used = "methods+results"
        text = "\n\n".join(parts)

    elif abstract:
        used = "abstract_only"
        text = "Abstract:\n" + abstract

    else:
        used = "fallback_fulltext"
        text = raw_text

    text = text[:MAX_CHARS_PER_PAPER].strip()
    return text, used


In [ ]:
# =========================
# BLOCK(2) SCREEN & CLASSIFY PAPERS
#    - Local LLM (Ollama) OR GPT model
# =========================

# ---------- ROUTING CONFIG ----------
OLLAMA_URL = "http://127.0.0.1:11434/api/chat"
OLLAMA_MODEL = "llama3:latest"  # change if needed

OPENAI_MODEL = "gpt-5.2"
client = OpenAI()  # uses OPENAI_API_KEY env var

OUT_CSV = "eeg_hybrid_results.csv"

# =========================
# GPT RUBRIC PROMPT (Full rubric)
# =========================
RUBRIC = r"""
You are an expert scientific screening and classification system for EEG papers.
Your task is to classify one scientific paper into exactly one of the following categories based strictly on the Methods and Results sections (do not rely on the abstract alone unless no other text is available). You must not infer, assume, or guess. Only use explicitly stated information in the provided text.

🎯 Classification Categories (Final Output Label) Choose one and only one:
1. Irrelevant → The paper fails any relevance criterion in Step 1.
2. Relevant-without-psychometrics → The paper passes Step 1, but does not report psychometric properties of single-trial EEG estimates.
3. Relevant-with-psychometrics → The paper passes Step 1 and explicitly reports psychometric properties of single-trial EEG estimates.

🧪 Step 1: Relevance Screening (MANDATORY) The paper must satisfy ALL:
1) Single-trial EEG reporting:
A. Single-trial / within-person variability:
"Single-trial" OR "Single trial" OR "Within-person" OR "Within person" OR "Within-subject" OR "Within subject" OR
"Trial-wise" OR "Trial wise" OR "Trial-by-trial" OR "Trial by trial" OR "Varia*" OR "Vary*" OR "Fluctuation*" OR "Intra*"
B. EEG / electrophysiology:
"EEG" OR "ERP" OR "Event-related potential" OR "Evoked potential" OR "Electroencephalogra*" OR "Electrophysiology"
Important constraints:
• “EEG” and “ERP” must appear as stand-alone tokens or within parentheses
• Do not count substrings (e.g., “eeg” inside another word)

2) Human participants: explicitly human
3) Non-clinical sample: explicitly healthy / non-patient

If any Step 1 criterion fails → Irrelevant and record which criteria failed.

🧠 Step 2: Psychometrics Screening (ONLY if Step 1 passes)
If psychometric keywords appear anywhere in Methods or Results, mark psychometrics_reported true and list keywords.
Important note: do not decide whether psychometrics refer to EEG or questionnaires yet. If keyword appears anywhere in Methods or Results include and flag for manual checking.

Psychometrics keywords include:
Reliability, Validity, Test theory, Test-retest, Parallel-forms, Split-half, Odd-even, Internal consistency, Consistency,
Intraclass correlation, ICC, Cronbach, Generalizability theory, G-theory, Generalizability coefficient, G-coefficient,
G-study, D-study, Dependability, Index of dependability, Variance components, Variance decomposition, Stability,
Invariance, Non-invariance, Measurement error, Error components, Latent state trait theory, LSTT, Empirical decomposition,
Specificity, Spearman-Brown, Bland-Altman, Construct validity, Content validity, Convergent validity, Discriminant validity,
Factorial validity, Criterion validity, Differential item functioning, Dimensionality, Multidimensional, Discrimination power, Difficulty

🗂️ Required Output Format (STRICT JSON)
Return ONLY valid JSON. No extra text.

{
 "final_classification": "Irrelevant | Relevant-without-psychometrics | Relevant-with-psychometrics",
 "step1_relevance": {
   "single_trial_EEG": true,
   "human_participants": true,
   "non_clinical_sample": true,
   "failed_criteria": []
 },
 "step2_psychometrics": {
   "psychometrics_reported": true,
   "matched_keywords": ["ICC", "Test-retest"]
 },
 "evidence_notes": {
   "single_trial_EEG_evidence": "exact sentence or phrase",
   "human_sample_evidence": "exact sentence or phrase",
   "non_clinical_evidence": "exact sentence or phrase",
   "psychometrics_evidence": "exact sentence or phrase or 'not reported'"
 }
}

🚫 Hard Rules:
- Do not infer intent or methodology
- Do not use background knowledge
- Do not normalize or paraphrase evidence
- If information is unclear → treat as not reported
- Always prefer false negatives over false positives
""".strip()


In [ ]:
# =========================
# BLOCK (3) ASSIGN LABELS: Irrelevant vs Relevant (prompt logic)
#     + hard keyword verification for Step 1 evidence
# =========================

# =========================
# Schemas
# =========================
OLLAMA_STEP1_SCHEMA = {
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "single_trial_EEG": {"type": "boolean"},
        "human_participants": {"type": "boolean"},
        "non_clinical_sample": {"type": "boolean"},
        "failed_criteria": {"type": "array", "items": {"type": "string"}},
        "evidence": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "single_trial_EEG_evidence": {"type": "string"},
                "human_sample_evidence": {"type": "string"},
                "non_clinical_evidence": {"type": "string"},
            },
            "required": ["single_trial_EEG_evidence", "human_sample_evidence", "non_clinical_evidence"],
        },
    },
    "required": ["single_trial_EEG", "human_participants", "non_clinical_sample", "failed_criteria", "evidence"],
}

OPENAI_SCHEMA = {
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "final_classification": {
            "type": "string",
            "enum": ["Irrelevant", "Relevant-without-psychometrics", "Relevant-with-psychometrics"],
        },
        "step1_relevance": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "single_trial_EEG": {"type": "boolean"},
                "human_participants": {"type": "boolean"},
                "non_clinical_sample": {"type": "boolean"},
                "failed_criteria": {"type": "array", "items": {"type": "string"}},
            },
            "required": ["single_trial_EEG", "human_participants", "non_clinical_sample", "failed_criteria"],
        },
        "step2_psychometrics": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "psychometrics_reported": {"type": "boolean"},
                "matched_keywords": {"type": "array", "items": {"type": "string"}},
            },
            "required": ["psychometrics_reported", "matched_keywords"],
        },
        "evidence_notes": {
            "type": "object",
            "additionalProperties": False,
            "properties": {
                "single_trial_EEG_evidence": {"type": "string"},
                "human_sample_evidence": {"type": "string"},
                "non_clinical_evidence": {"type": "string"},
                "psychometrics_evidence": {"type": "string"},
            },
            "required": [
                "single_trial_EEG_evidence",
                "human_sample_evidence",
                "non_clinical_evidence",
                "psychometrics_evidence",
            ],
        },
    },
    "required": ["final_classification", "step1_relevance", "step2_psychometrics", "evidence_notes"],
}

# =========================
# Keyword checks (hard verification)
# =========================
SINGLE_TRIAL_PATTERNS = [
    r"\bsingle[-\s]?trial\b",
    r"\btrial[-\s]?by[-\s]?trial\b",
    r"\btrial[-\s]?wise\b",
    r"\bwithin[-\s]?person\b",
    r"\bwithin[-\s]?subject\b",
    r"\bintra\w*\b",
    r"\bfluctuation\w*\b",
    r"\bvariab\w*\b",
    r"\bvary\w*\b",
]

EEG_ERP_TOKEN = re.compile(r"(?i)(?:\bEEG\b|\bERP\b|\(EEG\)|\(ERP\))")

def has_single_trial(text: str) -> bool:
    t = text.lower()
    return any(re.search(p, t) for p in SINGLE_TRIAL_PATTERNS)

def has_eeg_token(text: str) -> bool:
    return EEG_ERP_TOKEN.search(text) is not None or re.search(r"(?i)\belectroencephalograph\w*\b", text) is not None

def looks_human(text: str) -> bool:
    t = text.lower()
    return any(x in t for x in ["participants", "subjects", "healthy adults", "humans", "volunteers", "students", "years old", "men", "women"])

def looks_non_clinical(text: str) -> bool:
    t = text.lower()
    return any(x in t for x in ["healthy", "non-clinical", "nonclinical", "no history of", "no psychiatric", "control group"])

def evidence_is_verifiable(step1: Dict[str, Any]) -> bool:
    ev = step1.get("evidence", {})
    st = ev.get("single_trial_EEG_evidence", "") or ""
    hu = ev.get("human_sample_evidence", "") or ""
    nc = ev.get("non_clinical_evidence", "") or ""

    if not (has_single_trial(st) and has_eeg_token(st)):
        return False
    if not looks_human(hu):
        return False
    if not looks_non_clinical(nc):
        return False
    return True


In [ ]:
# =========================
# BLOCK (4) REVIEW & ANALYZE RESULTS
#     - Call models
#     - Decide route
#     - Build rows for CSV
# =========================

# =========================
# Ollama Step1 prompt + call
# =========================
OLLAMA_STEP1_PROMPT = r"""
You are an expert screening system for EEG papers.
Task: Perform ONLY Step 1 (relevance) using the provided text. Do not infer or guess.

Step 1 passes only if ALL are explicitly supported:
1) Single-trial EEG reporting: provide evidence containing BOTH:
   - a single-trial / trial-by-trial / within-person / within-subject / intra- / variability / fluctuation concept
   - AND an EEG/ERP/electroencephalography/electrophysiology term
   Note: EEG/ERP must appear as standalone token or in parentheses.
2) Human participants: explicit.
3) Non-clinical sample: explicit healthy/non-patient.

Output MUST match the JSON schema exactly.

Return:
- single_trial_EEG (boolean)
- human_participants (boolean)
- non_clinical_sample (boolean)
- failed_criteria: list of strings among:
  ["single_trial_EEG", "human_participants", "non_clinical_sample"] that are false.
- evidence:
  - single_trial_EEG_evidence: exact sentence/phrase (must include both single-trial concept and EEG/ERP/etc)
  - human_sample_evidence: exact sentence/phrase
  - non_clinical_evidence: exact sentence/phrase

Text:
{paper_text}
""".strip()

def ollama_step1(paper_text: str) -> Dict[str, Any]:
    payload = {
        "model": OLLAMA_MODEL,
        "messages": [
            {"role": "system", "content": "You are a strict JSON generator. Output only valid JSON."},
            {"role": "user", "content": OLLAMA_STEP1_PROMPT.format(paper_text=paper_text)},
        ],
        "stream": False,
        "format": OLLAMA_STEP1_SCHEMA,
        "options": {"temperature": 0}
    }

    r = requests.post(OLLAMA_URL, json=payload, timeout=300)
    if r.status_code != 200:
        raise RuntimeError(f"Ollama HTTP {r.status_code}: {r.text[:300]}")

    data = r.json()
    content = data["message"]["content"].strip()
    return json.loads(content)

# =========================
# OpenAI calls
# =========================
def openai_full_classify(paper_text: str) -> Dict[str, Any]:
    resp = client.responses.create(
        model=OPENAI_MODEL,
        instructions=RUBRIC,
        input=f"Paper text:\n{paper_text}",
        text={
            "format": {
                "type": "json_schema",
                "name": "EEGScreeningResult",
                "schema": OPENAI_SCHEMA,
                "strict": True
            }
        },
    )
    return json.loads(resp.output_text)

def openai_step2_only(paper_text: str, step1: Dict[str, Any]) -> Dict[str, Any]:
    prompt = f"""
You must TRUST the following Step 1 results as already verified by code and do NOT re-evaluate Step 1.
Use them exactly as given and fill in Step 2 (psychometrics) + final_classification accordingly.
Return STRICT JSON with the same schema.

Verified Step 1 (do not change):
{json.dumps({
  "single_trial_EEG": step1["single_trial_EEG"],
  "human_participants": step1["human_participants"],
  "non_clinical_sample": step1["non_clinical_sample"],
  "failed_criteria": step1["failed_criteria"],
  "evidence": step1["evidence"]
}, ensure_ascii=False)}

Paper text:
{paper_text}
""".strip()

    resp = client.responses.create(
        model=OPENAI_MODEL,
        instructions="Return ONLY valid JSON that matches the schema exactly.",
        input=prompt,
        text={
            "format": {
                "type": "json_schema",
                "name": "EEGScreeningResult",
                "schema": OPENAI_SCHEMA,
                "strict": True
            }
        },
    )

    out = json.loads(resp.output_text)

    out["step1_relevance"] = {
        "single_trial_EEG": step1["single_trial_EEG"],
        "human_participants": step1["human_participants"],
        "non_clinical_sample": step1["non_clinical_sample"],
        "failed_criteria": step1["failed_criteria"],
    }
    out["evidence_notes"]["single_trial_EEG_evidence"] = step1["evidence"]["single_trial_EEG_evidence"]
    out["evidence_notes"]["human_sample_evidence"] = step1["evidence"]["human_sample_evidence"]
    out["evidence_notes"]["non_clinical_evidence"] = step1["evidence"]["non_clinical_evidence"]

    return out

# =========================
# Utility functions (for CSV columns)
# =========================
def yn(flag: Any) -> str:
    if flag is None:
        return ""
    if isinstance(flag, bool):
        return "Y" if flag else "N"
    s = str(flag).strip().lower()
    if s in {"y", "yes", "true", "1"}:
        return "Y"
    if s in {"n", "no", "false", "0"}:
        return "N"
    return str(flag)

def is_sent_to_gpt(route: str) -> str:
    if not route:
        return ""
    return "Y" if ("GPT_" in route or "__GPT_" in route) else "N"

def derive_irrelevant_reason(failed_criteria: List[str]) -> str:
    if not failed_criteria:
        return ""
    mapping = {
        "single_trial_EEG": "Not single-trial EEG (or missing EEG/ERP token + single-trial concept)",
        "human_participants": "Human participants not explicitly stated",
        "non_clinical_sample": "Non-clinical/healthy sample not explicitly stated",
    }
    return "; ".join(mapping.get(x, x) for x in failed_criteria)

def split_psychometric_keywords(out: Dict[str, Any]) -> Tuple[str, str]:
    matched = out.get("step2_psychometrics", {}).get("matched_keywords", []) or []
    if not isinstance(matched, list):
        matched = [str(matched)]

    final_cls = out.get("final_classification", "")
    psycho_reported = out.get("step2_psychometrics", {}).get("psychometrics_reported", False)

    keywords_main = ", ".join(map(str, matched)) if matched else ""

    if psycho_reported and final_cls != "Relevant-with-psychometrics":
        keywords_unrelated = keywords_main
    else:
        keywords_unrelated = ""

    return keywords_main, keywords_unrelated


In [ ]:
# =========================
# BLOCK (5) SAVE FILTERED OUTPUT (CSV)
# =========================

def main():
    pdfs = sorted(glob.glob(os.path.join(PDF_DIR, "*.pdf")))
    if not pdfs:
        raise RuntimeError(f"No PDFs found in {PDF_DIR}. Upload PDFs to /content/pdfs in Colab.")

    rows: List[Dict[str, Any]] = []

    for path in tqdm(pdfs, desc="Hybrid classify"):
        fname = os.path.basename(path)

        rec: Dict[str, Any] = {
            "title": "",
            "file": fname,
            "used_text_type": "",
            "route": "",
            "final_classification": "",
            "Relevant Y/N": "",
            "Irrelevant Reason": "",
            "Psychometrics Y/N": "",
            "Keywords": "",
            "Psychometric keywords mentioned in methods or results section unrelated to EEG quality": "",
            "sent_to_gpt (Y/N)": "",
            "gpt_output_json": "",
        }
        rec["title"] = extract_title_from_pdf(path)

        try:
            raw = extract_text_from_pdf(path, MAX_PAGES_TO_READ)
            sections = extract_AMR(raw)
            model_input, used = build_model_input(sections, raw)
            rec["used_text_type"] = used

            if not model_input:
                raise ValueError("Empty extracted text")

            # --- Step 1 via Ollama ---
            step1 = ollama_step1(model_input)

            # If Ollama output not verifiable -> borderline -> full GPT
            if not evidence_is_verifiable(step1):
                rec["route"] = "GPT_full_borderline"
                out = openai_full_classify(model_input)

            else:
                # Verified Step1; accept Ollama for Step1
                if not (step1["single_trial_EEG"] and step1["human_participants"] and step1["non_clinical_sample"]):
                    rec["route"] = "Ollama_verified_Irrelevant"
                    out = {
                        "final_classification": "Irrelevant",
                        "step1_relevance": {
                            "single_trial_EEG": step1["single_trial_EEG"],
                            "human_participants": step1["human_participants"],
                            "non_clinical_sample": step1["non_clinical_sample"],
                            "failed_criteria": step1["failed_criteria"],
                        },
                        "step2_psychometrics": {"psychometrics_reported": False, "matched_keywords": []},
                        "evidence_notes": {
                            "single_trial_EEG_evidence": step1["evidence"]["single_trial_EEG_evidence"],
                            "human_sample_evidence": step1["evidence"]["human_sample_evidence"],
                            "non_clinical_evidence": step1["evidence"]["non_clinical_evidence"],
                            "psychometrics_evidence": "not reported",
                        }
                    }
                else:
                    rec["route"] = "Ollama_verified_Step1__GPT_Step2"
                    out = openai_step2_only(model_input, step1)

            # --- Fill CSV fields ---
            rec["final_classification"] = out["final_classification"]
            rec["Relevant Y/N"] = "Y" if out["final_classification"] != "Irrelevant" else "N"

            failed = out.get("step1_relevance", {}).get("failed_criteria", []) or []
            rec["Irrelevant Reason"] = (
                derive_irrelevant_reason(failed)
                if out["final_classification"] == "Irrelevant"
                else ""
            )

            rec["Psychometrics Y/N"] = yn(out.get("step2_psychometrics", {}).get("psychometrics_reported", False))

            kw_main, kw_unrel = split_psychometric_keywords(out)
            rec["Keywords"] = kw_main
            rec["Psychometric keywords mentioned in methods or results section unrelated to EEG quality"] = kw_unrel

            rec["sent_to_gpt (Y/N)"] = is_sent_to_gpt(rec["route"])
            rec["gpt_output_json"] = json.dumps(out, ensure_ascii=False)

        except Exception as e:
            rec["route"] = "ERROR"
            rec["final_classification"] = "ERROR"
            rec["Relevant Y/N"] = ""
            rec["Irrelevant Reason"] = f"ERROR: {str(e)}"
            rec["Psychometrics Y/N"] = ""
            rec["Keywords"] = ""
            rec["Psychometric keywords mentioned in methods or results section unrelated to EEG quality"] = ""
            rec["sent_to_gpt (Y/N)"] = "N"
            rec["gpt_output_json"] = ""

        rows.append(rec)

    cols = [
        "title",
        "file",
        "used_text_type",
        "route",
        "final_classification",
        "Relevant Y/N",
        "Irrelevant Reason",
        "Psychometrics Y/N",
        "Keywords",
        "Psychometric keywords mentioned in methods or results section unrelated to EEG quality",
        "sent_to_gpt (Y/N)",
        "gpt_output_json",
    ]

    pd.DataFrame(rows, columns=cols).to_csv(OUT_CSV, index=False, encoding="utf-8")
    print(f"✅ Done. Saved {OUT_CSV}")

# Run
main()
